In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import BertForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score, f1_score
import transformers
import random

# -----------------------
# Set random seed for reproducibility
seed_val = 42
torch.manual_seed(seed_val)
np.random.seed(seed_val)
random.seed(seed_val)

# -----------------------
# Patch torch.get_default_device in transformers
def patched_get_default_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def patch_transformers():
    import transformers.trainer
    if hasattr(transformers.trainer, 'torch'):
        original_torch_generator = transformers.trainer.torch.Generator
        class PatchedGenerator(original_torch_generator):
            def __new__(cls, *args, **kwargs):
                instance = super(PatchedGenerator, cls).__new__(cls, *args, **kwargs)
                instance.get_default_device = patched_get_default_device
                return instance
        transformers.trainer.torch.Generator = PatchedGenerator

patch_transformers()

# -----------------------
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------
# Load the dataset
dataset_path = 'datasets/train.csv'
df = pd.read_csv(dataset_path)
print("Dataset sample:")
print(df.head())

# -----------------------
# Task 2: Define distillation functions
def distill_bert_weights_odd(teacher: nn.Module, student: nn.Module) -> nn.Module:
    if hasattr(teacher, "bert"):
        teacher_layers = teacher.bert.encoder.layer
        student_layers = student.bert.encoder.layer
        for i in range(len(student_layers)):
            student_layers[i].load_state_dict(teacher_layers[2*i + 1].state_dict())
    return student

def distill_bert_weights_even(teacher: nn.Module, student: nn.Module) -> nn.Module:
    if hasattr(teacher, "bert"):
        teacher_layers = teacher.bert.encoder.layer
        student_layers = student.bert.encoder.layer
        for i in range(len(student_layers)):
            student_layers[i].load_state_dict(teacher_layers[2*i].state_dict())
    return student


c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



Using device: cpu
Dataset sample:
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [2]:
# Task 3: Preprocess dataset
teacher_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["comment_text"], padding="max_length", truncation=True, max_length=128)
    tokenized_inputs["labels"] = [int(x) for x in examples["toxic"]]
    return tokenized_inputs

# Convert to Hugging Face Dataset and take a subset for faster iteration
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
subset_size = 1000  # Adjust as needed
tokenized_dataset = tokenized_dataset.select(range(min(len(tokenized_dataset), subset_size)))
tokenized_dataset = tokenized_dataset.remove_columns(["comment_text", "id", "obscene", "identity_hate", "threat", "severe_toxic", "insult"])
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"], device=device)

# Split dataset into training and evaluation sets
train_size = int(0.8 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

# -----------------------
# Task 2: Odd Layer vs Even Layer Training
# Initialize teacher model
teacher_model = BertForSequenceClassification.from_pretrained(teacher_model_name, num_labels=2)

# Configure the student model with 6 layers
student_config = teacher_model.config
student_config.num_hidden_layers = 6

# Initialize student models for odd and even layer distillation
student_model_odd = BertForSequenceClassification(student_config).to(device)
student_model_even = BertForSequenceClassification(student_config).to(device)

# Distill the models from the teacher
student_model_odd = distill_bert_weights_odd(teacher_model, student_model_odd)
student_model_even = distill_bert_weights_even(teacher_model, student_model_even)

# -----------------------
# Task 3: LoRA (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"],
)
student_model_lora = get_peft_model(BertForSequenceClassification(student_config).to(device), lora_config)
student_model_lora.print_trainable_parameters()

# -----------------------
# Training arguments with mixed precision and adjusted hyperparameters
training_args = TrainingArguments(
    output_dir="./results_lora",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,            # Increased epochs for better convergence
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    remove_unused_columns=False,
    no_cuda=not torch.cuda.is_available(),
    fp16=torch.cuda.is_available(),
    label_names=["labels"]
)

# -----------------------
# Trainer for LoRA model
trainer_lora = Trainer(
    model=student_model_lora,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Starting LoRA model training...")
trainer_lora.train()

Map: 100%|██████████| 159571/159571 [00:18<00:00, 8701.06 examples/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 221,184 || all params: 67,177,730 || trainable%: 0.3293
Starting LoRA model training...


Epoch,Training Loss,Validation Loss
1,0.307000,0.361396
2,0.320800,0.355269
3,0.352000,0.355224
4,0.314200,0.354657
5,0.383000,0.353245


c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\utils\other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1e2c84ac-f5ba-4048-83d4-6c876f37ad0f)') - silently ignoring the lookup for the file config.json in bert-base-uncased.
  warnings.warn(
c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\utils\save_and_load.py:246: UserWarning: Could not find a config file in bert-base-uncased - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=250, training_loss=0.33424978637695313, metrics={'train_runtime': 373.9381, 'train_samples_per_second': 10.697, 'train_steps_per_second': 0.669, 'total_flos': 133146875904000.0, 'train_loss': 0.33424978637695313, 'epoch': 5.0})

In [3]:
# Task 4: Evaluation and Analysis
def evaluate_model(model, dataset, threshold=0.5):
    """Evaluate the model: compute accuracy, F1-score, and optionally adjust threshold using softmax probabilities."""
    trainer = Trainer(model=model, args=training_args, eval_dataset=dataset)
    predictions = trainer.predict(dataset)
    logits = predictions.predictions
    labels = predictions.label_ids
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
    
    # Optionally use thresholding on toxic probability (assumes label 1 = Toxic)
    preds = (probs[:, 1] >= threshold).astype(int)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    
    # Debug: print average toxic probability for evaluation set
    avg_toxic_prob = probs[:, 1].mean()
    print(f"Average toxic probability on eval set: {avg_toxic_prob:.3f}")
    
    return acc, f1

# Evaluate models
acc_odd, f1_odd = evaluate_model(student_model_odd, eval_dataset)
acc_even, f1_even = evaluate_model(student_model_even, eval_dataset)
acc_lora, f1_lora = evaluate_model(student_model_lora, eval_dataset)

# Also evaluate loss using Trainer.evaluate
trainer_odd = Trainer(model=student_model_odd, args=training_args, eval_dataset=eval_dataset)
trainer_even = Trainer(model=student_model_even, args=training_args, eval_dataset=eval_dataset)
trainer_lora_model = Trainer(model=student_model_lora, args=training_args, eval_dataset=eval_dataset)

loss_odd = trainer_odd.evaluate().get("eval_loss")
loss_even = trainer_even.evaluate().get("eval_loss")
loss_lora = trainer_lora_model.evaluate().get("eval_loss")

print("\n📊 **Model Evaluation Results**")
print(f"🔹 **Odd Layer Model**  → Loss: {loss_odd:.4f}, Accuracy: {acc_odd:.4f}, F1-score: {f1_odd:.4f}")
print(f"🔹 **Even Layer Model** → Loss: {loss_even:.4f}, Accuracy: {acc_even:.4f}, F1-score: {f1_even:.4f}")
print(f"🔹 **LoRA Model**      → Loss: {loss_lora:.4f}, Accuracy: {acc_lora:.4f}, F1-score: {f1_lora:.4f}")

Average toxic probability on eval set: 0.609


Average toxic probability on eval set: 0.651


Average toxic probability on eval set: 0.083



📊 **Model Evaluation Results**
🔹 **Odd Layer Model**  → Loss: 0.8867, Accuracy: 0.1150, F1-score: 0.0237
🔹 **Even Layer Model** → Loss: 0.9866, Accuracy: 0.1150, F1-score: 0.0237
🔹 **LoRA Model**      → Loss: 0.3532, Accuracy: 0.8850, F1-score: 0.8310


In [5]:
# Save models using torch.save (state_dict)
torch.save(student_model_odd.state_dict(), "model/student_model_odd.pth")
torch.save(student_model_even.state_dict(), "model/student_model_even.pth")
torch.save(student_model_lora.state_dict(), "model/student_model_lora.pth")